In [23]:
import mysql.connector
import time
import logging
from mysql.connector import Error

# Database configuration
MYSQL_CONFIG = {
    'host': 'mariadb',
    'database': 'bsky_db',
    'user': 'bsky_user',
    'password': 'bsky_password',
    'port': 3306
}

class MySQLDB:
    def __init__(self, config, max_retries=15):
        self.config = config
        self.max_retries = max_retries
        self.base_delay = 1  # Start with 1 second delay

    def get_connection(self):
        """Get database connection with retry logic"""
        for attempt in range(self.max_retries):
            try:
                conn = mysql.connector.connect(**self.config)
                if conn.is_connected():
                    return conn
            except Error as e:
                if attempt == self.max_retries - 1:
                    print(f"Failed to connect after {self.max_retries} attempts. Last error: {e}")
                    raise e
                
                # Exponential backoff with jitter
                delay = self.base_delay * (2 ** attempt) + (attempt * 0.1)
                print(f"Connection attempt {attempt + 1} failed: {e}. Retrying in {delay:.1f} seconds...")
                time.sleep(delay)
        
        raise Exception(f"Could not establish connection after {self.max_retries} attempts")

    def fetch_all(self, query, params=()):
        """Execute query with connection retry logic"""
        for attempt in range(self.max_retries):
            conn = None
            try:
                conn = self.get_connection()
                cur = conn.cursor(dictionary=True)
                cur.execute(query, params)
                result = cur.fetchall()
                cur.close()
                conn.close()
                return result
                
            except Error as e:
                if conn and conn.is_connected():
                    conn.close()
                
                if attempt == self.max_retries - 1:
                    print(f"Query failed after {self.max_retries} attempts. Last error: {e}")
                    raise e
                
                # Exponential backoff
                delay = self.base_delay * (2 ** attempt) + (attempt * 0.1)
                print(f"Query attempt {attempt + 1} failed: {e}. Retrying in {delay:.1f} seconds...")
                time.sleep(delay)
            
            except Exception as e:
                if conn and conn.is_connected():
                    conn.close()
                print(f"Unexpected error: {e}")
                raise e
        
        raise Exception(f"Query could not be executed after {self.max_retries} attempts")

    def get_posts_by_author_did(self, author_did):
        query = "SELECT * FROM posts WHERE author_did = %s"
        return self.fetch_all(query, (author_did,))

    def get_posts_by_user(self, user_handle):
        query = "SELECT * FROM posts WHERE author_handle = %s"
        return self.fetch_all(query, (user_handle,))

    def search_posts_by_text(self, search_term):
        query = "SELECT * FROM posts WHERE text LIKE %s"
        wildcard = f"%{search_term}%"
        return self.fetch_all(query, (wildcard,))
        
    def get_posts_by_author_handle(self, author_handle):
        query = "SELECT * FROM posts WHERE author_handle = %s"
        return self.fetch_all(query, (author_handle,))
        
    def close(self):
        # Connection is closed automatically in fetch_all
        pass

In [24]:
data = MySQLDB(MYSQL_CONFIG, max_retries=15)
print("Database connection initialized with retry logic (15 attempts max)")

Database connection initialized with retry logic (15 attempts max)


In [3]:
# Test database connection and get some basic stats
stats_query = """
SELECT 
    COUNT(*) as total_posts,
    COUNT(DISTINCT author_did) as unique_authors,
    COUNT(CASE WHEN author_handle IS NOT NULL THEN 1 END) as resolved_handles,
    MIN(created_at) as oldest_post,
    MAX(created_at) as newest_post
FROM posts
"""

stats = data.fetch_all(stats_query)
print("Database Statistics:")
for stat in stats:
    print(f"  Total Posts: {stat['total_posts']}")
    print(f"  Unique Authors: {stat['unique_authors']}")
    print(f"  Resolved Handles: {stat['resolved_handles']}")
    print(f"  Resolution Rate: {(stat['resolved_handles']/stat['total_posts']*100):.1f}%" if stat['total_posts'] > 0 else "N/A")
    print(f"  Oldest Post: {stat['oldest_post']}")
    print(f"  Newest Post: {stat['newest_post']}")

Database Statistics:
  Total Posts: 81704
  Unique Authors: 40660
  Resolved Handles: 81691
  Resolution Rate: 100.0%
  Oldest Post: 2019-05-22 04:33:02
  Newest Post: 2025-07-08 15:55:59


In [4]:
# words of sadness
words_of_sadness = [
    "sad", "unhappy", "depressed", "downcast", "sorrow", "died", "death", "cry", "tears",
    "grief", "mourning", "heartbroken", "loss", "tragic", "sorrowful", "despair",
    "melancholy", "gloomy", "dismal", "woeful", "forlorn", "desolate",
    "distress", "anguish", "pain", "suffering", "misery", "heartache",
    "lament", "weep", "sigh", "wail", "sob", "lamentation", "bereavement"
]
def find_sad_posts():
    sad_posts = []
    for word in words_of_sadness:
        posts = data.search_posts_by_text(word)
        sad_posts.extend(posts)
    return sad_posts

sad_posts = find_sad_posts()
for post in sad_posts:
    print(f"User: {post['author_handle']}, Content: {post['text']}, Date: {post['created_at']}")

User: wukbelav.bsky.social, Content: Y'know, one of these days i would like the ability to play fighting games again without wanting to put a bullet in my skull but sadly today is not that day, Date: 2025-07-08 01:13:27
User: teawhilewriting.bsky.social, Content: I got on an indie game dev list for posting on indiegames but sadly I know not the first thing about coding (although I sure wish I could get in on video game scriptwriting). Am solely a player and fan 😀, Date: 2025-07-08 01:13:37
User: basilsunflower.is.eepy.now, Content: *vuelvo del salón y la luz de mi cuarto no se enciende*

yo: qué le ha pasado a la puta bombilla?

literalmente la puta bombilla:, Date: 2025-07-08 01:13:16
User: cathy12341234.bsky.social, Content: Omg i can't sleep 😴 I am thinking about a lot so really sad but I be ok I am going to listen to some folk music soon, Date: 2025-07-08 01:13:42
User: sugarcubedog.bsky.social, Content: Feels more like trump’s personal army of sadists., Date: 2025-07-08 01:13:46
U

In [5]:
# Word Cloud Generation in Notebook
import matplotlib.pyplot as plt
from wordcloud import WordCloud
import re
from collections import Counter

# Comprehensive stop words
STOP_WORDS = {
    'a', 'an', 'and', 'are', 'as', 'at', 'be', 'by', 'for', 'from', 'has', 'he', 
    'in', 'is', 'it', 'its', 'of', 'on', 'that', 'the', 'to', 'was', 'were', 
    'will', 'with', 'the', 'this', 'but', 'they', 'have', 'had', 'what', 'said', 
    'each', 'which', 'she', 'do', 'how', 'their', 'if', 'up', 'out', 'many', 
    'then', 'them', 'these', 'so', 'some', 'her', 'would', 'make', 'like', 'into', 
    'him', 'time', 'two', 'more', 'very', 'when', 'come', 'may', 'say', 'get', 
    'use', 'your', 'way', 'about', 'than', 'first', 'been', 'call', 'who', 'oil', 
    'sit', 'now', 'find', 'long', 'down', 'day', 'did', 'get', 'come', 'made', 
    'may', 'part', 'over', 'new', 'sound', 'take', 'only', 'little', 'work', 
    'know', 'place', 'year', 'live', 'me', 'back', 'give', 'most', 'very', 'after',
    'don', 'can', 'said', 'go', 'see', 'no', 'could', 'people', 'my', 'one',
    'rt', 'via', 'http', 'https', 'www', 'com', 'bsky', 'social', 'at',
    'really', 'actually', 'literally', 'basically', 'totally', 'definitely', 
    'probably', 'maybe', 'perhaps', 'seems', 'looks', 'feels', 'sounds',
    'something', 'someone', 'somewhere', 'somehow', 'sometimes', 'anything', 
    'anyone', 'anywhere', 'anytime', 'everything', 'everyone', 'everywhere',
    'nothing', 'nobody', 'nowhere', 'never', 'always', 'often', 'usually',
    'yes', 'no', 'ok', 'okay', 'well', 'oh', 'ah', 'um', 'uh', 'er', 'hmm'
}

def clean_text_for_wordcloud(text):
    """Clean text for word cloud generation"""
    text = text.lower()
    # Remove URLs
    text = re.sub(r'http[s]?://\S+', '', text)
    # Remove mentions and hashtags
    text = re.sub(r'@\w+', '', text)
    text = re.sub(r'#\w+', '', text)
    # Remove special characters
    text = re.sub(r'[^\w\s]', ' ', text)
    # Remove extra whitespace
    text = ' '.join(text.split())
    return text

def create_notebook_wordcloud(search_term=None, hours_ago=24, colormap='viridis'):
    """Create a word cloud in the notebook"""
    
    # Get posts
    if search_term:
        query = """
        SELECT text FROM posts 
        WHERE created_at > DATE_SUB(NOW(), INTERVAL %s HOUR)
        AND text LIKE %s
        AND LENGTH(text) > 10
        """
        posts = data.fetch_all(query, (hours_ago, f'%{search_term}%'))
    else:
        query = """
        SELECT text FROM posts 
        WHERE created_at > DATE_SUB(NOW(), INTERVAL %s HOUR)
        AND LENGTH(text) > 10
        """
        posts = data.fetch_all(query, (hours_ago,))
    
    if not posts:
        print("No posts found!")
        return
    
    print(f"Found {len(posts)} posts")
    
    # Clean and combine text
    all_text = []
    for post in posts:
        cleaned = clean_text_for_wordcloud(post['text'])
        if len(cleaned.strip()) > 5:
            all_text.append(cleaned)
    
    combined_text = ' '.join(all_text)
    
    # Create word cloud
    wordcloud = WordCloud(
        width=1200, 
        height=600,
        max_words=150,
        background_color='white',
        colormap=colormap,
        stopwords=STOP_WORDS,
        relative_scaling=0.5,
        collocations=False
    ).generate(combined_text)
    
    # Display
    plt.figure(figsize=(15, 8))
    plt.imshow(wordcloud, interpolation='bilinear')
    plt.axis('off')
    
    title = f"Word Cloud"
    if search_term:
        title += f" - '{search_term}' posts"
    title += f" (last {hours_ago} hours)"
    
    plt.title(title, fontsize=16, fontweight='bold', pad=20)
    plt.tight_layout()
    plt.show()
    
    # Show top words
    words = combined_text.split()
    word_freq = Counter(word for word in words if word not in STOP_WORDS and len(word) > 2)
    
    print(f"\nTop 15 most frequent words:")
    for i, (word, count) in enumerate(word_freq.most_common(15), 1):
        print(f"   {i:2d}. {word:<15} ({count} times)")

# Example usage:
print("Word cloud functions loaded! Use:")
print("create_notebook_wordcloud()  # All posts")
print("create_notebook_wordcloud('love', colormap='plasma')  # Love posts with plasma colors")

Word cloud functions loaded! Use:
create_notebook_wordcloud()  # All posts
create_notebook_wordcloud('love', colormap='plasma')  # Love posts with plasma colors


In [6]:
#clear jupyter output
import time
from IPython.display import clear_output
clear_output()
#posts per user top posters, get their top keyworkds per user

top_users_query = """
SELECT author_handle, COUNT(*) as post_count
FROM posts
WHERE created_at > NOW() - INTERVAL 1 HOUR
GROUP BY author_handle
ORDER BY post_count DESC
LIMIT 5
"""
while True:
    top_users = data.fetch_all(top_users_query)
    clear_output()
    print("\nTop 5 users by post count:")
    for user in top_users:
        print(f"User: {user['author_handle']}, Posts: {user['post_count']}")    
        #get top keywords for this user
        user_posts = data.get_posts_by_user(user['author_handle'])
        user_text = ' '.join(post['text'] for post in user_posts)
        user_words = re.findall(r'\b\w+\b', user_text.lower())
        user_word_freq = Counter(word for word in user_words if word not in STOP_WORDS and len(word) > 2)
        print(f"  Top keywords: {', '.join(word for word, count in user_word_freq.most_common(20))}")   
        print() 
    time.sleep(1)  # Pause to avoid overwhelming the output



Top 5 users by post count:
User: longtail.news, Posts: 461
  Top keywords: feed, july, 2025, monday, news, published, latest, tuesday, ukraine, world, ukrainian, national, unn, wire, jalopnik, west, times, today, oregonlive, india

User: floofypuppy.bsky.social, Posts: 260
  Top keywords: woof, roof, ruff, arf, grr, bork, yip

User: new3rd.bsky.social, Posts: 247
  Top keywords: 東京都, 大阪府, 大阪市, 中央区, 千代田区, 愛知県, 新宿区, 渋谷区, 神奈川県, が登場, 品川区, 2025, 福岡県, 京都府, fit, 名古屋市, 福岡市, summer, を発表, が開催

User: gjkhgvvvyy.bsky.social, Posts: 207
  Top keywords: beg, you, help, ahmed, kiss, head, days, left, save, son, life, tired, donating, sharing, campaign, pinned, page, mjtothemax, jim7654, swordandquill

User: movies-on-sale.bsky.social, Posts: 105
  Top keywords: available, genre, cast, trailer, reviews, movie, today, rottentomatoes, itunes, audio, ray, off, amazon, blu, imdb, comedy, germany, john, dolby, action



KeyboardInterrupt: 

In [11]:
#right wing call words
right_wing_words = [
    # Core conservative values
    "patriot", "freedom", "liberty", "constitution", "constitutional", "founding fathers",
    "conservative", "republican", "GOP", "right wing", "traditional values", "family values",
    "christian values", "moral values", "religious freedom", "faith", "god", "prayer",
    
    # Trump & MAGA movement
    "maga", "trump", "trump 2024", "america first", "make america great",
    "drain the swamp", "deep state", "establishment", "swamp", "fake news",
    "mainstream media", "MSM", "liberal media", "biased media",
    
    # Nationalism & patriotism
    "nationalism", "patriotism", "american exceptionalism", "flag", "anthem",
    "pledge of allegiance", "national pride", "america", "USA", "united states",
    
    # Government & politics
    "small government", "limited government", "states rights", "federalism",
    "constitutional rights", "bill of rights", "separation of powers",
    "law and order", "rule of law", "individual rights", "personal responsibility",
    
    # Economic conservatism
    "free market", "capitalism", "free enterprise", "entrepreneur", "business owner",
    "job creator", "economic freedom", "lower taxes", "tax cuts", "deregulation",
    "fiscal responsibility", "balanced budget", "national debt", "spending cuts",
    
    # Gun rights
    "2nd amendment", "second amendment", "gun rights", "pro-gun", "NRA",
    "right to bear arms", "self defense", "concealed carry", "open carry",
    "gun owner", "firearm", "shall not be infringed",
    
    # Military & law enforcement
    "military", "armed forces", "veteran", "support our troops", "police",
    "law enforcement", "blue lives matter", "back the blue", "thin blue line",
    "national defense", "strong defense", "peace through strength",
    
    # Immigration & border
    "border security", "illegal immigration", "border wall", "immigration control",
    "secure the border", "deportation", "illegal alien", "sanctuary cities",
    "chain migration", "merit based immigration",
    
    # Social issues
    "pro-life", "right to life", "unborn", "sanctity of life", "abortion",
    "traditional marriage", "nuclear family", "parental rights", "school choice",
    "homeschool", "religious liberty", "conscience rights",
    
    # Anti-left sentiments
    "anti-socialism", "anti-communist", "anti-marxist", "anti-woke", "woke",
    "cancel culture", "political correctness", "virtue signaling", "identity politics",
    "cultural marxism", "critical race theory", "CRT", "leftist", "liberal",
    "progressive agenda", "radical left", "socialist agenda",
    
    # Sovereignty & anti-globalism
    "national sovereignty", "anti-globalism", "america first", "globalist",
    "new world order", "UN", "international law", "foreign aid",
    "trade deficit", "fair trade", "energy independence",
    
    # Populist themes
    "drain the swamp", "corrupt politicians", "career politicians", "term limits",
    "government accountability", "transparency", "taxpayer", "hard working americans",
    "forgotten americans", "silent majority", "common sense", "real americans",
    
    # Conspiracy & fringe terms
    "deep state", "shadow government", "mainstream narrative", "alternative facts",
    "sovereign citizen", "constitutionalist", "militia", "three percenter",
    "don't tread on me", "come and take it", "molon labe",
    
    # Modern conservative media
    "tucker carlson", "fox news", "newsmax", "OAN", "breitbart", "daily wire",
    "podcast", "alternative media", "citizen journalist", "truth",
    
    # Election & voting
    "election integrity", "voter ID", "stop the steal", "rigged election",
    "election fraud", "ballot harvesting", "voter fraud", "audit",
    
    # Cultural issues
    "western civilization", "judeo-christian", "heritage", "tradition",
    "cultural decline", "moral decay", "family breakdown", "gender ideology",
    "transgender", "pronouns", "biological sex", "women's sports"
]
def find_right_wing_posts():
    right_wing_posts = []
    for word in right_wing_words:
        posts = data.search_posts_by_text(word)
        right_wing_posts.extend(posts)
    return right_wing_posts
right_wing_posts = find_right_wing_posts()
stats = {
    'most common words': Counter(),
    'total posts': 0,
    'unique authors': set(),
    'total users': 0,
}
for post in right_wing_posts:
    stats['total posts'] += 1
    stats['unique authors'].add(post['author_did'])
    words = re.findall(r'\b\w+\b', post['text'].lower())
    stats['most common words'].update(word for word in words if word not in STOP_WORDS and len(word) > 2)
stats['total users'] = len(stats['unique authors'])
print(f"Found {stats['total posts']} right-wing posts from {stats['total users']} unique users")
print("Most common words:")
for word, count in stats['most common words'].most_common(20):
    print(f"  {word}: {count} times")

Found 24272 right-wing posts from 13864 unique users
Most common words:
  you: 5095 times
  trump: 4374 times
  not: 2986 times
  just: 2354 times
  all: 2304 times
  his: 1668 times
  there: 1414 times
  because: 1108 times
  our: 1062 times
  que: 1037 times
  think: 965 times
  even: 907 times
  news: 873 times
  god: 820 times
  2025: 807 times
  america: 806 times
  why: 760 times
  maga: 744 times
  around: 743 times
  good: 738 times


In [10]:
left_wing_words = [
    # Core progressive values
    "equality", "justice", "progressive", "social justice", "equity", "fairness",
    "liberation", "solidarity", "community", "collective action", "grassroots",
    
    # Economic justice
    "living wage", "minimum wage", "workers rights", "union", "unionize", "strike",
    "collective bargaining", "wealth inequality", "income inequality", "tax the rich",
    "universal basic income", "UBI", "affordable housing", "rent control",
    "student debt", "free college", "free tuition", "public education funding",
    
    # Environmental & climate
    "climate change", "climate crisis", "climate action", "environment", "green",
    "renewable energy", "solar", "wind power", "sustainability", "carbon tax",
    "fossil fuels", "green new deal", "environmental justice", "clean energy",
    
    # Healthcare & social services
    "universal healthcare", "medicare for all", "single payer", "public option",
    "mental health", "reproductive rights", "bodily autonomy", "abortion access",
    
    # Social justice & civil rights
    "diversity", "inclusion", "LGBTQ+", "LGBTQIA", "transgender rights", "gay rights",
    "feminism", "feminist", "women's rights", "gender equality", "pay gap",
    "anti-racism", "racial justice", "black lives matter", "BLM", "systemic racism",
    "police reform", "criminal justice reform", "prison abolition", "restorative justice",
    "voter rights", "voting rights", "gerrymandering", "democracy reform",
    
    # Immigration & refugee rights
    "immigration reform", "refugee rights", "DACA", "dreamers", "sanctuary cities",
    "border policy", "family separation", "detention centers",
    
    # Anti-establishment & system change
    "anti-capitalism", "anti-imperialism", "anti-fascism", "antifa", "resist",
    "revolution", "systemic change", "democratic socialism", "socialism",
    "capitalism critique", "corporate power", "big pharma", "wall street",
    
    # International & peace
    "anti-war", "peace", "diplomacy", "human rights", "international law",
    "global south", "decolonization", "indigenous rights", "land back",
    
    # Modern progressive terms
    "intersectionality", "privilege", "allyship", "mutual aid", "harm reduction",
    "reparations", "defund police", "abolition", "prison industrial complex",
    "food justice", "housing justice", "climate justice", "economic justice",
    "disability rights", "accessibility", "neurodiversity"
]
def find_left_wing_posts():
    left_wing_posts = []
    for word in left_wing_words:
        posts = data.search_posts_by_text(word)
        left_wing_posts.extend(posts)
    return left_wing_posts
left_wing_posts = find_left_wing_posts()
stats = {
    'most common words': Counter(),
    'total posts': 0,
    'unique authors': set(),
    'total users': 0,
}
for post in left_wing_posts:
    stats['total posts'] += 1
    stats['unique authors'].add(post['author_did'])
    words = re.findall(r'\b\w+\b', post['text'].lower())
    stats['most common words'].update(word for word in words if word not in STOP_WORDS and len(word) > 2)
stats['total users'] = len(stats['unique authors'])
print(f"Found {stats['total posts']} left-wing posts from {stats['total users']} unique users")
print("Most common words:")
for word, count in stats['most common words'].most_common(20):
    print(f"  {word}: {count} times")

Found 2249 left-wing posts from 1796 unique users
Most common words:
  you: 484 times
  peace: 357 times
  not: 268 times
  all: 254 times
  community: 232 times
  just: 211 times
  trump: 206 times
  prize: 202 times
  our: 182 times
  nobel: 160 times
  green: 138 times
  his: 125 times
  there: 115 times
  justice: 111 times
  climate: 107 times
  change: 102 times
  here: 91 times
  war: 91 times
  because: 90 times
  want: 90 times


In [ ]:
from tqdm import tqdm

# Analyze each user's post history based on the keywords for right and left wing and determine how many total right and left wing users we have
def analyze_user_post_history(user_posts):
    user_stats = {
        'right wing': 0,
        'left wing': 0
    }
    for post in user_posts:
        if any(word in post['text'].lower() for word in right_wing_words):
            user_stats['right wing'] += 1
        if any(word in post['text'].lower() for word in left_wing_words):
            user_stats['left wing'] += 1
    return user_stats
# Analyze all users
user_analysis = {}
for user in tqdm(data.fetch_all("SELECT DISTINCT author_handle FROM posts")):
    user_posts = data.get_posts_by_author_handle(user['author_handle'])
    user_analysis[user['author_handle']] = analyze_user_post_history(user_posts)
# Count total right and left wing users
total_right_wing_users = sum(1 for stats in user_analysis.values() if stats['right wing'] > 0)
total_left_wing_users = sum(1 for stats in user_analysis.values() if stats['left wing'] > 0)
print(f"Total right-wing users: {total_right_wing_users}")
print(f"Total left-wing users: {total_left_wing_users}")    

 35%|███▍      | 18533/53244 [00:51<02:13, 259.11it/s]

Connection attempt 1 failed: 2003 (HY000): Can't connect to MySQL server on 'mariadb:3306' (99). Retrying in 1.0 seconds...


 36%|███▌      | 19044/53244 [00:53<01:10, 483.56it/s]

Connection attempt 1 failed: 2003 (HY000): Can't connect to MySQL server on 'mariadb:3306' (99). Retrying in 1.0 seconds...
Connection attempt 2 failed: 2003 (HY000): Can't connect to MySQL server on 'mariadb:3306' (99). Retrying in 2.1 seconds...
Connection attempt 3 failed: 2003 (HY000): Can't connect to MySQL server on 'mariadb:3306' (99). Retrying in 4.2 seconds...
Connection attempt 4 failed: 2003 (HY000): Can't connect to MySQL server on 'mariadb:3306' (99). Retrying in 8.3 seconds...
